In [32]:
import pandas as pd
from datetime import datetime
from random import randint
from tqdm import tqdm
import numpy as np

In [47]:
new_data=  pd.read_pickle('MinHash.pkl')

## Part 1 : MinHash

In [ ]:
new_data = pd.read_pickle('processed.pkl')

In [ ]:
def hash_date(value):
    return int(pd.Timestamp(value).timestamp())

In [ ]:
def hash_string(value,p):
    m =2^32 -1
    return sum([ ord(x)*(p^i) for i,x in enumerate(value)])%m

In [ ]:
def hash_float(value):
    integer,decimal  = str(value).split('.')
    return int(integer+decimal)

In [ ]:
def minHash(customer,p,permutations):
    vec  = [0 for i in range(len(customer)) ]
    for i,val in enumerate(customer):
        if not isinstance(val,int):
            if isinstance(val,datetime): 
                val  = hash_date(val)
            elif isinstance(val,str): 
                val = hash_string(val,p)
            elif isinstance(val,float): 
                val = hash_float(val)

        mini = float('inf')
        for perm_vals in permutations:
            a,b = perm_vals
            output = (a*val +b)%p

            if(mini > output):
                mini  = output

        vec[i] = output
    return vec
    

In [ ]:
p  = 4993
max_val = 2^32-1
N  = new_data.shape[1]
permutations = [ (randint(0,max_val),randint(0,max_val)) for i in range(N)]
new_data['minhash']  = [minHash([*x[1]],p,permutations) for x in tqdm(new_data.iterrows())]

985322it [02:36, 6298.64it/s]
